# FPL Deploy Model
Now we have trained the model, we need to deploy it to serve predictions.

WIP Note: We'll worry about tuning the model later, let's just get it deployed first.

In [1]:
import os
from google.cloud import aiplatform

In [2]:
PROJECT = !gcloud config get-value project
PROJECT = PROJECT[0]
BUCKET = PROJECT
REGION = "europe-west1"
MODEL_DISPLAYNAME = "mid_model_20230805_164422"

print(f"MODEL_DISPLAYNAME: {MODEL_DISPLAYNAME}")

# from https://cloud.google.com/vertex-ai/docs/predictions/pre-built-containers
SERVING_CONTAINER_IMAGE_URI = (
    "us-docker.pkg.dev/vertex-ai/prediction/tf2-cpu.2-3:latest"
)

os.environ["BUCKET"] = BUCKET
os.environ["REGION"] = REGION

MODEL_DISPLAYNAME: mid_model_20230805_164422


In [3]:
uploaded_model = aiplatform.Model.upload(
    display_name=MODEL_DISPLAYNAME,
    artifact_uri=f"gs://{BUCKET}/fpl/trained-models/{MODEL_DISPLAYNAME}/savedmodel",
    serving_container_image_uri=SERVING_CONTAINER_IMAGE_URI,
)

Creating Model
Create Model backing LRO: projects/343566520815/locations/us-central1/models/8615942440043413504/operations/8254931747116941312
Model created. Resource name: projects/343566520815/locations/us-central1/models/8615942440043413504@1
To use this Model in another session:
model = aiplatform.Model('projects/343566520815/locations/us-central1/models/8615942440043413504@1')


In [4]:
MACHINE_TYPE = "e2-standard-2"

endpoint = uploaded_model.deploy(
    machine_type=MACHINE_TYPE,
    accelerator_type=None,
    accelerator_count=None,
)

Creating Endpoint
Create Endpoint backing LRO: projects/343566520815/locations/us-central1/endpoints/8310262814358896640/operations/3255936160735690752
Endpoint created. Resource name: projects/343566520815/locations/us-central1/endpoints/8310262814358896640
To use this Endpoint in another session:
endpoint = aiplatform.Endpoint('projects/343566520815/locations/us-central1/endpoints/8310262814358896640')
Deploying model to Endpoint : projects/343566520815/locations/us-central1/endpoints/8310262814358896640
Deploy Endpoint model backing LRO: projects/343566520815/locations/us-central1/endpoints/8310262814358896640/operations/4093605691426603008
Endpoint model deployed. Resource name: projects/343566520815/locations/us-central1/endpoints/8310262814358896640


In [8]:
instance = [
    {
        "minutes": [90],
        "goals_scored": [2],
        "assists": [1],
        "clean_sheets": [1],
        "penalties_missed": [0],
        "bps": [25],
        "yellow_threshold": [0],
        "red_cards": [0],
        "own_goals": [0],
        "influence": [12.5],
        "creativity": [8.7],
        "threat": [20.1],
        "start_cost": [8.0],
        "end_cost": [8.5],
    }
]

In [9]:
endpoint.predict(instance)

Prediction(predictions=[[8.836689]], deployed_model_id='6236907140791926784', model_version_id='1', model_resource_name='projects/343566520815/locations/us-central1/models/8615942440043413504', explanations=None)

# Cleanup

Delete endpoint to avoid extra charges.

In [10]:
endpoint.undeploy_all()

Undeploying Endpoint model: projects/343566520815/locations/us-central1/endpoints/8310262814358896640
Undeploy Endpoint model backing LRO: projects/343566520815/locations/us-central1/endpoints/8310262814358896640/operations/666788587462721536
Endpoint model undeployed. Resource name: projects/343566520815/locations/us-central1/endpoints/8310262814358896640


resource name: projects/343566520815/locations/us-central1/endpoints/8310262814358896640

In [11]:
endpoint.delete()

Deleting Endpoint : projects/343566520815/locations/us-central1/endpoints/8310262814358896640
Delete Endpoint  backing LRO: projects/343566520815/locations/us-central1/operations/7278072840442609664
Endpoint deleted. . Resource name: projects/343566520815/locations/us-central1/endpoints/8310262814358896640
